In [1]:
import pandas as pd
import numpy as ny
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype=int)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset[["Purchased"]].value_counts()

Purchased
0            257
1            143
Name: count, dtype: int64

In [7]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [8]:
Independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]

In [9]:
Independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [10]:
Independent.shape

(400, 3)

In [11]:
Dependent=dataset[["Purchased"]]

In [12]:
Dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [13]:
from sklearn.model_selection import train_test_split,GridSearchCV
x_train,x_test,y_train,y_test=train_test_split(Independent,Dependent,test_size=0.3,random_state=0)

In [14]:
from sklearn.naive_bayes import CategoricalNB
param_grid={'alpha':[1,10,100],'force_alpha':[True],'fit_prior':[True],'class_prior':[None]}
grid=GridSearchCV(CategoricalNB(),param_grid,cv=5,refit=True,n_jobs=-1,verbose=3)

In [15]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


C:\Users\91805\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan]
  warnings.warn(
C:\Users\91805\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [1, 10, 100], 'class_prior': [None],
                         'fit_prior': [True], 'force_alpha': [True]},
             verbose=3)

In [16]:
y_pred=grid.predict(x_test)

In [17]:
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score

In [18]:
cm=confusion_matrix(y_test,y_pred)

In [19]:
print(cm)

[[76  3]
 [ 9 32]]


In [20]:
clf_rpt=classification_report(y_test,y_pred)

In [21]:
print (clf_rpt)

              precision    recall  f1-score   support

           0       0.89      0.96      0.93        79
           1       0.91      0.78      0.84        41

    accuracy                           0.90       120
   macro avg       0.90      0.87      0.88       120
weighted avg       0.90      0.90      0.90       120



In [22]:
roc_score=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

In [23]:
roc_score

0.9235875270145106

In [24]:
re=grid.cv_results_

In [25]:
print ("Best parameters are :", grid.best_params_)

Best parameters are : {'alpha': 1, 'class_prior': None, 'fit_prior': True, 'force_alpha': True}


In [26]:
table=pd.DataFrame.from_dict(re)

In [27]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.032710,0.006816,0.007578,0.001046,1,None,True,True,"{'alpha': 1, 'class_prior': None, 'fit_prior':...",0.750000,0.714286,0.750000,0.821429,NaN,NaN,NaN,1
1,0.030239,0.007320,0.017345,0.023035,10,None,True,True,"{'alpha': 10, 'class_prior': None, 'fit_prior'...",0.660714,0.696429,0.642857,0.660714,NaN,NaN,NaN,1
2,0.032504,0.001987,0.006811,0.002453,100,None,True,True,"{'alpha': 100, 'class_prior': None, 'fit_prior...",0.642857,0.642857,0.642857,0.625000,NaN,NaN,NaN,1
